## Module 1: Explaining Prediction

This demo gives an example of using Shapley values to explain sample-level,
local predictions. Given a classifier and a specific sample, we will investigate
which features are most relevant for making the final classification.  Our
example comes from the original Lundberg and Lee paper, and it is also the main
demo from the `shap` package. 


To run this code, you will need to install the `shap` and `lightgbm` packages. These are included in the iisa310 conda environment defined in this [yaml file](https://github.com/krisrs1128/talks/blob/master/2024/20241230/examples/environment-iisa310.yaml). If you use conda, you can download this file and run
```
conda env create -yf environment-iisa310.yaml
```
to install all the needed packages, though you can of course also install them
all manually.

In [1]:
import lightgbm as lgb
import shap

Specifically, we will analyze the results of a gradient boosting classifier
applied to an income prediction problem, available through the `shap` package
(and originally from the UCI repository). The classifier aims to see whether
income levels can be classified using 12 basic demographic features. Here are
the first few rows that will be use in the classifier. There are 32561 people in
the dataset overall.

In [ ]:
X, y = shap.datasets.adult()
X.head()

Here is the response, a binary indicator of whether the person in that rows makes more than $50K a year.

In [ ]:
y[:5]

We can now fit a simple gradient boosting model. These hyperparameters could
surely be tuned (and we could use a test set/cross validation to select them),
but since our purpose is to understand explanations, we won't worry too much
about optimizing performance.

In [ ]:
data = lgb.Dataset(X, y)
model = lgb.train({"learning_rate": 0.05}, data, 1000)

The `shap` package uses an object oriented approach, where we first initialize
an `explainer` object to generate all the local explanations that we will
ultimately work with. The block below uses a `KernelExplainer`, which uses the
weighted regression perspective to estimate Shapley values without having to
work with all subsets of features. We have chosen to explain just 50 of the
original samples so that this runs more quickly. The final output is a 50
$\times$ 12 vector of shap values $\varphi_{i} \in \mathbb{R}^{12}$ giving the
attributions for all 50 samples.

In [ ]:
x_sample = shap.sample(X, 50)

# define a kernel explainer and extract the shapley values
explainer = #???
shap_values = #???

The `shap` package has some great visualization utilities. First, we can make a
"force plot" describing the attributions for an individual samples.
`shap.initjs()` is used to make sure that the visualization will be interactive.

In [ ]:
ix = 2 # which sample to explain?
shap.initjs()

### plot shap values for sample ix

One of the properties that makes Shapley values so convenient is that we can
exactly recover the original, complex model's predictions using only the
original attributions. Positive attributions to a feature mean that it is
responsible for the sample having a larger prediction probability than the
baseline average (negative has the analogous interpretation).

In [ ]:
y_hat = model.predict(x_sample)
print(y_hat[ix])
print(y_hat.mean() + sum(shap_values[ix]))

We can use the same function to visualize Shpaley values across the entire
collection of explained samples. The plot has sorted samples so that those with
similar attributions $\varphi_{i}$ are placed next to one another. This reveals
some interesting clustering structure in the explanations. Even though the model
is complex, there are groups of samples that have similar predictions for
similar reasons.

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X)